In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "f1w", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9143 | AUPR: 0.6057 | Acc: 94.78%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9167 | AUPR: 0.6203 | Acc: 95.00%
model(kernel=linear, C=1)
Valid:         AUC: 0.9164 | AUPR: 0.6204 | Acc: 94.99%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9167 | AUPR: 0.6203 | Acc: 95.00%
Internal test: AUC: 0.9182 | AUPR: 0.6451 | Acc: 94.82%
External test: AUC: 0.9168 | AUPR: 0.6053 | Acc: 94.56%


In [3]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7606 | AUPR: 0.3630 | Acc: 93.77%
model(n_neighbors=5)
Valid:         AUC: 0.7870 | AUPR: 0.4229 | Acc: 94.29%
model(n_neighbors=7)
Valid:         AUC: 0.8064 | AUPR: 0.4603 | Acc: 94.49%
best model(n_neighbors=7)
Valid:         AUC: 0.8064 | AUPR: 0.4603 | Acc: 94.49%
Internal test: AUC: 0.8168 | AUPR: 0.4681 | Acc: 94.18%
External test: AUC: 0.8116 | AUPR: 0.4704 | Acc: 94.42%


In [4]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8716 | AUPR: 0.4395 | Acc: 77.18%
best model()
Valid:         AUC: 0.8716 | AUPR: 0.4395 | Acc: 77.18%
Internal test: AUC: 0.8834 | AUPR: 0.4734 | Acc: 74.06%
External test: AUC: 0.8691 | AUPR: 0.4322 | Acc: 72.47%


In [5]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9173 | AUPR: 0.6469 | Acc: 95.27%
model(C=10.0)
Valid:         AUC: 0.9176 | AUPR: 0.6473 | Acc: 95.24%
model(C=100.0)
Valid:         AUC: 0.9174 | AUPR: 0.6468 | Acc: 95.22%
best model(C=10.0)
Valid:         AUC: 0.9176 | AUPR: 0.6473 | Acc: 95.24%
Internal test: AUC: 0.9207 | AUPR: 0.6767 | Acc: 95.01%
External test: AUC: 0.9171 | AUPR: 0.6367 | Acc: 95.01%


In [6]:
from sklearn.neural_network import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=1000,
    activation='relu',
    solver='sgd', nesterovs_momentum=True, momentum=0.9)
param_grid_list = [{
    # 'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
    'hidden_layer_sizes': [(50, 50)],
    'learning_rate_init': [0.0003, 0.001],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(50, 50), learning_rate_init=0.0003)
Valid:         AUC: 0.9059 | AUPR: 0.6173 | Acc: 95.13%
model(hidden_layer_sizes=(50, 50), learning_rate_init=0.001)
Valid:         AUC: 0.9151 | AUPR: 0.6421 | Acc: 95.30%
best model(hidden_layer_sizes=(50, 50), learning_rate_init=0.001)
Valid:         AUC: 0.9151 | AUPR: 0.6421 | Acc: 95.30%
Internal test: AUC: 0.9181 | AUPR: 0.6562 | Acc: 95.16%
External test: AUC: 0.9152 | AUPR: 0.6283 | Acc: 94.99%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8803 | AUPR: 0.5619 | Acc: 85.00%
model(criterion=log_loss)
Valid:         AUC: 0.8806 | AUPR: 0.5689 | Acc: 85.58%
model(criterion=entropy)
Valid:         AUC: 0.8806 | AUPR: 0.5689 | Acc: 85.58%
best model(criterion=log_loss)
Valid:         AUC: 0.8806 | AUPR: 0.5689 | Acc: 85.58%
Internal test: AUC: 0.8718 | AUPR: 0.5844 | Acc: 85.47%
External test: AUC: 0.8802 | AUPR: 0.5799 | Acc: 84.95%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9147 | AUPR: 0.6512 | Acc: 89.65%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9157 | AUPR: 0.6522 | Acc: 89.85%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9163 | AUPR: 0.6518 | Acc: 89.76%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6546 | Acc: 89.81%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9165 | AUPR: 0.6538 | Acc: 89.80%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9163 | AUPR: 0.6547 | Acc: 89.78%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6546 | Acc: 89.81%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9165 | AUPR: 0.6538 | Acc: 89.80%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9163 | AUPR: 0.6547 | Acc: 89.78%
best model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9165 | AUPR: 0.6538 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9210 | AUPR: 0.6467 | Acc: 95.18%
model(criterion=squared_error)
Valid:         AUC: 0.9211 | AUPR: 0.6467 | Acc: 95.19%
best model(criterion=squared_error)
Valid:         AUC: 0.9211 | AUPR: 0.6467 | Acc: 95.19%
Internal test: AUC: 0.9235 | AUPR: 0.6862 | Acc: 95.45%
External test: AUC: 0.9205 | AUPR: 0.6558 | Acc: 95.14%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9177 | AUPR: 0.6345 | Acc: 95.25%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9182 | AUPR: 0.6387 | Acc: 95.22%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9186 | AUPR: 0.6422 | Acc: 95.30%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9186 | AUPR: 0.6422 | Acc: 95.30%
Internal test: AUC: 0.9173 | AUPR: 0.6770 | Acc: 95.45%
External test: AUC: 0.9135 | AUPR: 0.6330 | Acc: 95.11%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)